# Importing data from ERA5 nc files

#### Necessary libraries

In [ ]:
from netCDF4 import Dataset
import pandas as pd
from datetime import date, timedelta

#### Import nc file

In [ ]:
path = "F:/ERA5/ERA.nc"
ds = Dataset(path, 'r')

#### Coordinates of ground stations

In [ ]:
input_latlon = {35.818:51.781,
                35.864:51.658,
                35.896:51.6,
                35.935:51.434,
                35.953:51.364,
                35.985:51.639,
                35.999:50.983,
                36.026:51.513,
                36.034:51.423,
                36.064:51.126,
                36.08:51.324,
                36.105:51.311,
                36.14:50.688,
                36.146:50.749,
                36.177:50.909,
                36.195:50.898,
                36.21:50.767,
                36.226:50.956}

#### Import data from nc file

In [ ]:
#Change it to name of variable you want to save.
#Check inside of nc file for the available variables.
ds_sw = ds['sd'][:]

ds_lat = ds['latitude'][:]
ds_lon = ds['longitude'][:]
ds_time = ds['time'][:]
list_time = ds_time.data.tolist()
          
time_table = []
lat_table = []
lon_table = []
sw_table = []

base = date(1900, 1, 1)

#### Main body of code

In [ ]:
for t in list_time:
    for key in input_latlon:
        
        final_time = base + timedelta(hours=t)
        time_table.append(final_time)    
        
        lat_table.append(key)
        lon_table.append(input_latlon[key])

        sq_diff_lat = (ds_lat - key)**2
        sq_diff_lon = (ds_lon - input_latlon[key])**2

        min_index_lat = sq_diff_lat.argmin()
        min_index_lon = sq_diff_lon.argmin()

        sw = ds_sw[list_time.index(t), min_index_lat ,min_index_lon]
        sw_table.append(sw)

#### Check the lenght of tables

In [ ]:
print(len(time_table))
print(len(lat_table))
print(len(lon_table))
print(len(sw_table))

#### Merge all data to a DataFrame

In [ ]:
all_table = {'Time':time_table[:],
             'Lat':lat_table[:],
             'Lon':lon_table[:],
            'SWE':sw_table[:]}

final_ds = pd.DataFrame(all_table)
final_ds["Time"] = pd.to_datetime(final_ds["Time"])

#### Means of daily values

In [ ]:
final_ds_group = final_ds.groupby(['Time','Lat','Lon'])['sw'].mean().reset_index()

#### Export DataFrame to a Excel file

In [ ]:
final_ds_group.to_excel("f:/era5_ds.xlsx",index=False)